In [27]:
import pandas as pd
import numpy as np
import math
import os

def printCategory(category_dict):
    for category in category_dict:
        print(category)
        if category_dict[category] is not None:
            sub_dic = category_dict[category]
            printCategory(sub_dic)
            
def createLevelColumns(df, levels, new_columns):
    for i in range(len(df)):
        
        df_level1, df_level2, df_level3 = "NULL","NULL","NULL"
        
        df_level1, nextLevels = findNotNullLevel(df, i, levels)
        if nextLevels is not None:
            df_level2, nextLevels = findNotNullLevel(df, i, nextLevels)
            if nextLevels is not None:
                df_level3, nextLevel = findNotNullLevel(df, i, nextLevels)
        
        most_specific_category = "NULL"
        
        if df_level3 != "NULL":
            df_level3 = df_level3 + "__" + df_level2 + "__" + df_level1
        if df_level2 != "NULL":
            df_level2 = df_level2 + "__" + df_level1
        
        if df_level3 != "NULL":
            most_specific_category = df_level3
        elif df_level2 != "NULL":
            most_specific_category = df_level2
        elif df_level1 != "NULL":
            most_specific_category = df_level1
        
        df.iloc[i, df.columns.get_loc('level1')] = df_level1
        df.iloc[i, df.columns.get_loc('level2')] = df_level2
        df.iloc[i, df.columns.get_loc('level3')] = df_level3
        df.iloc[i, df.columns.get_loc('mostSpecificCategory')] = most_specific_category
        
def findNotNullLevel(df, i, levels):
    for level in levels:
        if df.iloc[i][level] == "NULL":
            continue
        else:
            return df.iloc[i][level], levels[level]
    return "NULL", None

def unionTwoLists(list1, list2):
    for category in list1:
        if category not in list2:
            list2.append(category)
    return list2

def checkNULL(checked_list):
    for item in checked_list:
        if item == "NULL":
            print("Contains NULL")
            return
    print("Not contains NULL")

#### 导入数据 yearly and range
#### process为designed schema
1. GazetteerEconomy table

    1.1 Index - Id
    
    1.2 Gazetteer Code - gazetteerId
    
    1.3 Category - categoryId
    
    1.4 yearly - startYear = endYear
    
    1.5 2010 data - data
    
    1.6 Unit - unitId
    
2. EconomyCategory table
    2.1 Category Division1 Subdivision - Agri. Subdivision - Misc. Subdivision - Service Division2 Subdivision - Househould
    
3. UnitCategory

4. Gazetteer 

In [51]:
path = os.path.abspath(os.getcwd())
df = pd.read_csv(path + "/Data2/Education - Yearly.csv")
df2 = pd.read_csv(path + "/Data2/Education - Range.csv")
df = df.dropna(axis = 0, how = 'all')
df2 = df2.dropna(axis = 0, how = 'all')

In [52]:
print(df.columns)
print(df['Category'].astype('category').unique())
print(df['Division1'].astype('category').unique())
print(df['Subdivision'].astype('category').unique())

Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title', 'Category', 'Division1',
       'Subdivision', '1949', '1950', '1951', '1952', '1953', '1954', '1955',
       '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019'],
      dtype='object')
[小学在校生 Elementary School Students, 小学老师 Elementary School Teachers, 受教育程度 Highest Level of Education, 新入学生 - 大学 Initial Student Enrollment - College...]
Categories (4, object): [小学在校生 Elementary School Students, 小学老师 Elementary School Teachers, 受教育程度 H

In [53]:
# create new columns at df and df2
new_columns = ['level1', 'level2', 'level3', 'mostSpecificCategory', 'categoryId']

for column in new_columns:
    df[column] = None
df = df.where(df.notnull(), "NULL")

for column in new_columns:
    df2[column] = None
df2 = df2.where(df2.notnull(), "NULL")

# create dictionary records category levles
heading = {'Category': {'Division1':{'Subdivision': None}}}

printCategory(heading)


createLevelColumns(df, heading, new_columns)
createLevelColumns(df2, heading, new_columns)

Category
Division1
Subdivision


In [54]:
level1 = unionTwoLists([cat for cat in df['level1'].astype('category').unique()], 
                       [cat for cat in df2['level1'].astype('category').unique()] )

level2 = unionTwoLists([cat for cat in df['level2'].astype('category').unique()], 
                       [cat for cat in df2['level2'].astype('category').unique()] )

level3 = unionTwoLists([cat for cat in df['level3'].astype('category').unique()], 
                       [cat for cat in df2['level3'].astype('category').unique()] )

most_specific_category = unionTwoLists([cat for cat in df['mostSpecificCategory'].astype('category').unique()],
                                       [cat for cat in df2['mostSpecificCategory'].astype('category').unique()])

total_categories = unionTwoLists(level1, level2)
total_categories = unionTwoLists(level3, total_categories)

# get total categories
total_categories.sort()
checkNULL(total_categories)
print("total number of categories are " + str(len(total_categories)))
# temp = [ item.split('__', 1) for item in total_categories]

# get most specific category
most_specific_category.sort()
checkNULL(most_specific_category)
print("total number of recorded categories are " + str(len(most_specific_category)))

# create dict "dic_category_id" store { category_name : id}
dic_category_id = {}
count = 1
for category in total_categories:
    if category != "NULL" and category not in dic_category_id:
        dic_category_id[category] = count
        count = count + 1

Contains NULL
total number of categories are 20
Not contains NULL
total number of recorded categories are 13


In [55]:
total_categories.sort()
total_categories

len(df)

3509

In [56]:
# creat categoryId column at dataframe
df_categoryId = []
for i in range(len(df)):
    category = df.iloc[i]['mostSpecificCategory']
    if category in dic_category_id:
        df_categoryId.append(dic_category_id[category])
    else:
        print("Not recorded category for entity " + str(i))
        break;
df['categoryId'] = df_categoryId

df_categoryId = []
for i in range(len(df2)):
    category = df2.iloc[i]['mostSpecificCategory']
    if category in dic_category_id:
        df_categoryId.append(dic_category_id[category])
    else:
        print("Not recorded category for entity " + str(i))
        break;
df2['categoryId'] = df_categoryId

In [59]:
print(df.columns)
print(df2.columns)

Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title', 'Category', 'Division1',
       'Subdivision', '1949', '1950', '1951', '1952', '1953', '1954', '1955',
       '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', 'level1', 'level2', 'level3', 'mostSpecificCategory',
       'categoryId'],
      dtype='object')
Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title', 'Category', 'Division1',
       'Subdivision', 'Start Year', 'End Year', 'Data', 'level1', 'level2',
       'level3', 'most

In [61]:
# create economy_df
economy_df = pd.DataFrame(columns = ['gazetteerId', 'categoryId', 'startYear', 'endYear', 'data'])
years = [str(i) for i in range(1949,2020)]
dic_for_economy_df = {'gazetteerId':[], 'categoryId':[], 'startYear':[], 'endYear':[], 'data':[]}

# Process yearly data
for i in range(len(df)):# each row
    for year in years: # 1949 - 2019
        if df.iloc[i][year] != "NULL":
            dic_for_economy_df['gazetteerId'].append(df.iloc[i]['村志代码 Gazetteer Code'])
            dic_for_economy_df['categoryId'].append(df.iloc[i]['categoryId'])
            dic_for_economy_df['startYear'].append(int(year))
            dic_for_economy_df['endYear'].append(int(year))
            dic_for_economy_df['data'].append(df.iloc[i][year])
#             dic_for_economy_df['unitId'].append(dic_for_unitId[df.iloc[i]['Unit']])

# Process range data
for i in range(len(df2)):
    dic_for_economy_df['gazetteerId'].append(df2.iloc[i]['村志代码 Gazetteer Code'])
    dic_for_economy_df['categoryId'].append(df2.iloc[i]['categoryId'])
    dic_for_economy_df['startYear'].append(df2.iloc[i]['Start Year'])
    dic_for_economy_df['endYear'].append(df2.iloc[i]['End Year'])
    dic_for_economy_df['data'].append(df2.iloc[i]['Data'])
#     dic_for_economy_df['unitId'].append(dic_for_unitId[df.iloc[i]['Unit']])

for attribute in economy_df.columns:
    economy_df[attribute] = dic_for_economy_df[attribute]
    

economy_df.head()

,gazetteerId,categoryId,startYear,endYear,data
0,1.0,11,1980.0,1980.0,500
1,1.0,12,1980.0,1980.0,26
2,2.0,11,2007.0,2007.0,258
3,2.0,12,2007.0,2007.0,25
4,3.0,11,1959.0,1959.0,1000


In [63]:
# create economyCategory_df
economyCategory_df = pd.DataFrame(columns = ['id', 'name', 'parentId'])
dic_for_ecoCategorydf = {'id':[], 'name':[], 'parentId':[]}

for category in dic_category_id:
    child_parent = category.split('__', 1)
    name = child_parent[0]
    if len(child_parent) == 1:
        dic_for_ecoCategorydf['id'].append(dic_category_id[category])
        dic_for_ecoCategorydf['name'].append(name)
        dic_for_ecoCategorydf['parentId'].append("NULL")
    else:
        parentId = dic_category_id[child_parent[1]]
        dic_for_ecoCategorydf['id'].append(dic_category_id[category])
        dic_for_ecoCategorydf['name'].append(name)
        dic_for_ecoCategorydf['parentId'].append(parentId)
        
for attribute in economyCategory_df.columns:
    economyCategory_df[attribute] = dic_for_ecoCategorydf[attribute]
len(economyCategory_df)

19

In [65]:
# creat economyUnitCategory_df
# economyUnitCategory_df = pd.DataFrame(columns = ['id', 'name'])
# dic_for_economyUnitCategory_df = {'id':[], 'name':[]}

# for unit_name in dic_for_unitId:
#     dic_for_economyUnitCategory_df['id'].append(dic_for_unitId[unit_name])
#     dic_for_economyUnitCategory_df['name'].append(unit_name)
    
# for attribute in economyUnitCategory_df.columns:
#     economyUnitCategory_df[attribute] = dic_for_economyUnitCategory_df[attribute]
# len(economyUnitCategory_df)

In [68]:
economy_df.to_csv('education_df.csv', index = False, na_rep = "NULL")

In [69]:
economyCategory_df.to_csv('educationCategory.csv', index = False, na_rep = "NULL")

In [45]:
# economyUnitCategory_df.to_csv('economyUnitCategory_df.csv', index = False, na_rep = "NULL")